# Remove uncompilable file

## Load solidity file

In [94]:
import os
os.chdir("/home/hieuvd/lvdthieu/CodeGen/")

In [95]:
import pandas as pd
train_file = pd.read_parquet("./data/solfile/train_file.parquet", engine="fastparquet")
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167310 entries, 0 to 167309
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   file_name         167310 non-null  object
 1   file_address      167310 non-null  object
 2   source_code       167310 non-null  object
 3   abi               167310 non-null  object
 4   compiler_version  167310 non-null  object
 5   library           167310 non-null  object
dtypes: object(6)
memory usage: 7.7+ MB


In [96]:
test_file = pd.read_parquet("./data/solfile/test_file.parquet", engine="fastparquet")
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18594 entries, 0 to 18593
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   file_name         18594 non-null  object
 1   file_address      18594 non-null  object
 2   source_code       18594 non-null  object
 3   abi               18594 non-null  object
 4   compiler_version  18594 non-null  object
 5   library           18594 non-null  object
dtypes: object(6)
memory usage: 871.7+ KB


## Filter out file with "import" word

In [97]:
test_file = test_file[~(test_file["source_code"].str.contains("import", regex=False))]\
    [["file_name", "file_address", "source_code", "compiler_version"]]\
        .reset_index(drop=True)
                        
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13642 entries, 0 to 13641
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   file_name         13642 non-null  object
 1   file_address      13642 non-null  object
 2   source_code       13642 non-null  object
 3   compiler_version  13642 non-null  object
dtypes: object(4)
memory usage: 426.4+ KB


In [98]:
train_file = train_file[~(train_file["source_code"].str.contains("import", regex=False))]\
    [["file_name", "file_address", "source_code", "compiler_version"]]\
        .reset_index(drop=True)
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127351 entries, 0 to 127350
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   file_name         127351 non-null  object
 1   file_address      127351 non-null  object
 2   source_code       127351 non-null  object
 3   compiler_version  127351 non-null  object
dtypes: object(4)
memory usage: 3.9+ MB


## Standardize compiler_version name

In [99]:
train_file["compiler_version"] = train_file["compiler_version"].apply(lambda x: x.split('+')[0].split('-')[0][1:])
train_file["compiler_version"]

0         0.4.20
1         0.8.11
2         0.6.12
3          0.8.7
4         0.4.24
           ...  
127346    0.6.11
127347    0.5.17
127348    0.4.19
127349    0.6.12
127350    0.5.17
Name: compiler_version, Length: 127351, dtype: object

In [100]:
test_file["compiler_version"] = test_file["compiler_version"].apply(lambda x: x.split('+')[0].split('-')[0][1:])
test_file["compiler_version"]

0        0.4.12
1        0.4.25
2        0.6.12
3        0.6.11
4         0.7.0
          ...  
13637     0.5.9
13638    0.6.12
13639    0.6.12
13640     0.8.1
13641     0.6.6
Name: compiler_version, Length: 13642, dtype: object

## Filter compiler with version < 0.4.11

In [101]:
def filter(stdized_version):
    version = list(map(int, stdized_version.split('.')))
    if version[0] == 0:
        if version[1] == 4:
            if version[2] < 11:
                return False
        if version[2] < 4:
            return False
    return True

In [102]:
test_file = test_file[test_file["compiler_version"].apply(filter)].reset_index(drop=True)
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12078 entries, 0 to 12077
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   file_name         12078 non-null  object
 1   file_address      12078 non-null  object
 2   source_code       12078 non-null  object
 3   compiler_version  12078 non-null  object
dtypes: object(4)
memory usage: 377.6+ KB


In [103]:
train_file = train_file[train_file["compiler_version"].apply(filter)].reset_index(drop=True)
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113462 entries, 0 to 113461
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   file_name         113462 non-null  object
 1   file_address      113462 non-null  object
 2   source_code       113462 non-null  object
 3   compiler_version  113462 non-null  object
dtypes: object(4)
memory usage: 3.5+ MB


## Test compile

In [104]:
os.chdir("/home/hieuvd/lvdthieu/CodeGen")

In [105]:
compiler_versions = list(set(train_file["compiler_version"].tolist() + test_file["compiler_version"].tolist()))
len(compiler_versions)

58

In [106]:
compiler_config = ""
for version in compiler_versions:
    compiler_config += '{version: "' + version + '"},\n'
with open("compiler_config.txt", "w") as f:
    f.write(compiler_config)

In [107]:
for i in range(100):
    source = test_file.loc[i, "source_code"]
    with open(f"./hardhat/contracts/sample_{i}.sol", "w") as f:
        f.write(source)

In [108]:
os.system("pwd")

/home/hieuvd/lvdthieu/CodeGen


0

In [109]:
os.chdir("hardhat/contracts/")

In [ ]:
# Remove unsatisfied file because of fault in defining pragma
wrong_file_list = [10471, 10552, 11668, 11729, 11884, 1857, 2106, 3165, 3352, 4069, 4259, 4617, 5572, 5739, 
                   619, 6485, 6554, 6566, 6707, 7646, 8374, 8457, 8701, 9136, 9796, 9818]
for file in wrong_file_list:
    os.system(f"rm sample_{file}.sol")

In [124]:
%%bash
rm sample_29.sol

In [125]:
%%time
os.system("npx hardhat compile")

contracts/sample_10.sol: Warning: SPDX license identifier not provided in source file. Before publishing, consider adding a comment containing "SPDX-License-Identifier: <SPDX-License>" to each source file. Use "SPDX-License-Identifier: UNLICENSED" for non-open-source code. Please see https://spdx.org for more information.

contracts/sample_16.sol: Warning: SPDX license identifier not provided in source file. Before publishing, consider adding a comment containing "SPDX-License-Identifier: <SPDX-License>" to each source file. Use "SPDX-License-Identifier: UNLICENSED" for non-open-source code. Please see https://spdx.org for more information.

contracts/sample_25.sol: Warning: SPDX license identifier not provided in source file. Before publishing, consider adding a comment containing "SPDX-License-Identifier: <SPDX-License>" to each source file. Use "SPDX-License-Identifier: UNLICENSED" for non-open-source code. Please see https://spdx.org for more information.

contracts/sample_3.sol: W


CPU times: user 35.9 ms, sys: 308 ms, total: 344 ms
Wall time: 24 s


256

## Store datase

In [129]:
os.chdir("/home/hieuvd/lvdthieu/CodeGen")

In [130]:
test_file.to_parquet("./data/sol-file-v2/test_file.parquet", engine="fastparquet")
train_file.to_parquet("./data/sol-file-v2/train_file.parquet", engine="fastparquet")

In [127]:
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12078 entries, 0 to 12077
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   file_name         12078 non-null  object
 1   file_address      12078 non-null  object
 2   source_code       12078 non-null  object
 3   compiler_version  12078 non-null  object
dtypes: object(4)
memory usage: 377.6+ KB


In [132]:
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113462 entries, 0 to 113461
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   file_name         113462 non-null  object
 1   file_address      113462 non-null  object
 2   source_code       113462 non-null  object
 3   compiler_version  113462 non-null  object
dtypes: object(4)
memory usage: 3.5+ MB


## Experiment


In [ ]:
train_data["source_idx"].nunique()

,file_name,file_address,contract_name
0,LendingPoolCore,0x5766067108e534419ce13f05899bc3e3f4344948,LendingPoolCore
1,AToken,0xb124541127a0a657f056d9dd06188c4f1b0e5aab,LendingPoolCore
2,commonwealthCredit,0x26780078e07cc33f0323df626f42fa92971561d4,LendingPoolCore
3,LendingPool,0x017788dded30fdd859d295b90d4e41a19393f423,LendingPoolCore
4,LendingPool,0xc1ec30dfd855c287084bf6e14ae2fdd0246baf0d,LendingPoolCore
...,...,...,...
149112,MiniMeTokenController,0x29070e64d6a9ea9eec83ee83c88d0c51d7efd257,Controlled
149113,MiniMeTokenController,0x29070e64d6a9ea9eec83ee83c88d0c51d7efd257,MiniMeTokenFactory
149114,MiniMeTokenFactory,0x1308a7ec3e82bcd3b63ada5f5dc27586ad8605ba,Controlled
149115,MiniMeTokenFactory,0x1308a7ec3e82bcd3b63ada5f5dc27586ad8605ba,MiniMeTokenFactory


In [ ]:
test_data["source_idx"].nunique()

,file_name,file_address,contract_name
0,LendingPool,0x6d252baea75459ed0077410613c5f6e51cab4750,LendingPoolCore
1,RewardPool,0x676ac3e2e53a2d010ab66a2ab5af7e0ee5a4fc00,LendingPoolCore
2,TokenDistributor,0x7694ad4ebffa35acda2744b6da09430ffdc846b5,LendingPoolCore
3,Claims,0xcafeada57182b2de3a81b7d8d97a7a223ff9c978,ClaimsData
4,Claims,0xcafeada57182b2de3a81b7d8d97a7a223ff9c978,PoolData
...,...,...,...
15014,Snap,0x4c5813b8c6fbbac76caa148aaf8910f236b56fdf,Snap
15015,DecentraLife,0x845bc135aeaa96c6ae21fb802c3c7af73c57bd39,Context
15016,Snoodle,0x91fe7bafad87a519e0969edf876a92637c4cc375,Snoodle
15017,DecentralandTemplate,0xf4e8987ae1c7ae9561fc6d61496515c627dc4d7f,ERC1271Bytes


# Build test suite

In [ ]:
import pandas as pd
test = pd.read_parquet("./data/test/test.parquet", engine="fastparquet")

In [ ]:
test_file = pd.read_parquet("./data/solfile/test_file.parquet", engine="fastparquet")
test_file.info()

,file_name,file_address,source_code,abi,compiler_version,library
0,KKNK,0x39176c7d2fe75fbcd0bfaece2fd58472b15b5a53,/**\r\n *Submitted for verification at Ethersc...,"[{""constant"":true,""inputs"":[],""name"":""name"",""o...",v0.4.12+commit.194ff033,
1,DssSpell,0x8efe2c82bd31b67fa262c0d364773629f6ea828a,// hevm: flattened sources of src/DssSpell.sol...,"[{""inputs"":[],""stateMutability"":""nonpayable"",""...",v0.6.12+commit.27d51765,DssExecLib:c061edfc9c38912e0ad4f3922aa84ee11fb...
2,StacsToken,0x286708f069225905194673755f12359e6aff6fe1,pragma solidity 0.4.25;\r\n\r\ncontract ERC20B...,"[{""constant"":true,""inputs"":[],""name"":""name"",""o...",v0.4.25+commit.59dbf8f1,
3,Strategy,0xb374387a340e6aa7d78385c4a4aac6b425a685b0,contract Strategy is BaseStrategyInitializable...,"[{""inputs"":[{""internalType"":""address"",""name"":""...",v0.6.12+commit.27d51765,
4,hexswap3,0x05b55fd424765323aaefa26ae46d5a88c4e8abd2,pragma solidity >=0.4.22 <0.7.0;\r\n\r\nabstra...,"[{""inputs"":[],""stateMutability"":""nonpayable"",""...",v0.6.11+commit.5ef660b1,
...,...,...,...,...,...,...
18589,UniswapV2Pair,0x48a7c27fca93b3282fc403cfa849b0ac3fe26645,// SPDX-License-Identifier: GPL-3.0\r\n\r\npra...,"[{""inputs"":[],""stateMutability"":""nonpayable"",""...",v0.6.12+commit.27d51765,
18590,KISS,0xb518575fef12793c0d3b703f35a75b8c9e09cb71,// SPDX-License-Identifier: MIT\n\npragma soli...,"[{""inputs"":[],""stateMutability"":""nonpayable"",""...",v0.8.7+commit.e28d00a7,
18591,DTCASH,0x805b28b949fc66fb9bb73e5c029d5a06ec83f322,// SPDX-License-Identifier: MIT\r\n\r\npragma ...,"[{""inputs"":[],""stateMutability"":""nonpayable"",""...",v0.8.1+commit.df193b15,
18592,ShrimpCocks,0x9815c5a8626ec80352c3abf6f138a094d51f276d,pragma solidity ^0.6.12;\r\n\r\n// uncomment i...,"[{""inputs"":[],""stateMutability"":""nonpayable"",""...",v0.6.12+commit.27d51765,


In [ ]:
test["compiler"]

0        v0.4.12+commit.194ff033
1        v0.6.12+commit.27d51765
2        v0.4.25+commit.59dbf8f1
3        v0.4.25+commit.59dbf8f1
4        v0.4.25+commit.59dbf8f1
                  ...           
11776    v0.4.24+commit.e67f0147
11777     v0.5.9+commit.e560f70d
11778     v0.5.9+commit.e560f70d
11779     v0.5.9+commit.e560f70d
11780     v0.5.9+commit.e560f70d
Name: compiler, Length: 11781, dtype: object

In [ ]:
compilers = [compiler.split('+')[0].split('-')[0] for compiler in test["compiler"].tolist()]

In [ ]:
compilers

['v0.4.12',
 'v0.6.12',
 'v0.4.25',
 'v0.4.25',
 'v0.4.25',
 'v0.4.25',
 'v0.4.25',
 'v0.7.0',
 'v0.8.4',
 'v0.8.4',
 'v0.8.0',
 'v0.4.20',
 'v0.6.2',
 'v0.6.2',
 'v0.6.2',
 'v0.5.2',
 'v0.5.2',
 'v0.5.2',
 'v0.4.21',
 'v0.8.4',
 'v0.4.25',
 'v0.4.25',
 'v0.4.6',
 'v0.8.6',
 'v0.8.4',
 'v0.7.6',
 'v0.4.19',
 'v0.4.19',
 'v0.4.19',
 'v0.4.19',
 'v0.4.19',
 'v0.4.17',
 'v0.4.17',
 'v0.4.17',
 'v0.4.17',
 'v0.4.17',
 'v0.4.17',
 'v0.8.3',
 'v0.4.24',
 'v0.4.24',
 'v0.4.24',
 'v0.8.7',
 'v0.6.6',
 'v0.8.7',
 'v0.4.24',
 'v0.8.0',
 'v0.6.0',
 'v0.8.4',
 'v0.8.0',
 'v0.8.0',
 'v0.7.0',
 'v0.5.15',
 'v0.5.15',
 'v0.5.15',
 'v0.5.15',
 'v0.4.18',
 'v0.4.24',
 'v0.4.26',
 'v0.8.6',
 'v0.7.0',
 'v0.4.26',
 'v0.4.26',
 'v0.4.26',
 'v0.4.26',
 'v0.4.26',
 'v0.4.20',
 'v0.4.20',
 'v0.4.20',
 'v0.6.12',
 'v0.7.5',
 'v0.4.18',
 'v0.4.18',
 'v0.4.18',
 'v0.4.18',
 'v0.4.18',
 'v0.6.12',
 'v0.6.12',
 'v0.6.12',
 'v0.5.16',
 'v0.8.9',
 'v0.6.12',
 'v0.4.21',
 'v0.4.24',
 'v0.4.24',
 'v0.8.6',
 'v0.6.12'

In [ ]:
compilers = list(set(compilers))
print(len(compilers))
compiler_config = ""
for compiler in compilers:
    compiler_config += '{version: "' + compiler + '"},\n'
with open("sol_version.txt", "w") as f:
    f.write(compiler_config)

81
